# 작업 1유형

## 데이터 로드

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')
df.head(5)

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


## 각 구급 보고서 별 출동시각과 신고시각의 차이를 ‘소요시간’ 컬럼을 만들고 초(sec)단위로 구하고 소방서명 별 소요시간의 평균을 오름차순으로 정렬 했을때 3번째로 작은 소요시간의 값과 소방서명을  출력하라 ***

In [58]:

df['call'] = pd.to_datetime(df['신고일자'].astype(str) + df['신고시각'].astype(str).str.zfill(6))
df['start'] = pd.to_datetime(df['출동일자'].astype(str) + df['출동시각'].astype(str).str.zfill(6))
# print(help(pd.to_datetime))
df['diff'] = (df['start'] - df['call']).dt.total_seconds()
df.groupby('소방서명')['diff'].mean().sort_values().reset_index().loc[2]

소방서명    종로소방서
diff    175.5
Name: 2, dtype: object

## 데이터 로드

In [59]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')
df.head(5)

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


## 학교 세부유형이 일반중학교인 학교들 중 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터하여 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교를 찾아서 해당 학교의 교원수를  출력하라

In [75]:
df['교원당맡은학생수'] = df['일반학급_학생수_계']/df['교원수_총계_계']
print(df.loc[(df['학교세부유형']=='일반중학교')&(df['시도'] == df.loc[df['학교세부유형']=='일반중학교'].groupby('시도').size().sort_values(ascending=False).index[1])].sort_values('교원당맡은학생수',ascending=False)['교원수_총계_계'].iloc[0])

33


## 데이터 로드

In [76]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv')
df.head(5)

,범죄분류,절도,사기,배임,방화,폭행
0,2018년_1월,13280,25318,656,143,12186
1,2018년_2월,12504,19159,403,135,12087
2,2018년_3월,14733,24509,471,148,13841
3,2018년_4월,14781,23147,414,135,13973
4,2018년_5월,15988,23584,451,127,14117


## **5대 범죄(절도, 사기, 배임, 방화, 폭행)의 월별 총 발생건수를 총범죄수라고 표현하자. 18,19년의 각각 분기별 총범죄수의 월평균 값을 구했을때 최대값을 가지는 년도와 분기를 구하고 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수를  출력하라(1분기:1,2,3월 / 2분기 : 4,5,6월 / 3분기 7,8,9월 / 4분기 10,11,12월 , 1분기 월평균 : 1,2,3월의 총범죄수 평균) **

In [100]:
df['year'] = df['범죄분류'].str.split('년_').str[0]
df['month'] = df['범죄분류'].str.split('년_').str[1].str[:-1].astype(int)
df['sum'] = df['절도'] + df['사기'] + df['배임'] + df['방화'] + df['폭행'] 
def quater(row):
    if row in [1,2,3]:
        return  1
    elif row in [4,5,6]:
        return  2
    elif row in [7,8,9]:
        return  3
    else:
        return  4
df['quater'] = df['month'].apply(quater)
print(df.loc[(df['year'] == df.groupby(['year','quater'])['sum'].mean().sort_values(ascending = False).reset_index().loc[0]['year'])&(df['quater'] == df.groupby(['year','quater'])['sum'].mean().sort_values(ascending = False).reset_index().loc[0]['quater'])].sort_values('사기',ascending=False).iloc[0]['사기'])

27766


# 작업 2유형

## 예측 변수 General_Health, test.csv에 대해 ID별로 General_Health 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 General_Health 두개만 존재해야함. 평가지표는 f1score

In [110]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier 
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')

x = train.drop(['ID','General_Health'],axis=1)
y = train['General_Health']
z = test.drop(['ID',],axis=1)

combine = pd.concat([x,z])
combine = pd.get_dummies(combine)
x = combine[:len(x)]
z = combine[len(x):]

xtr,xte,ytr,yte = train_test_split(x,y,test_size= 0.33,random_state = 23)

RFC = RandomForestClassifier(random_state = 23)
RFC.fit(xtr,ytr)
pred = RFC.predict(xte)

f1 = f1_score(yte,pred,average= 'micro')
print(f1)

test['General_Health'] = RFC.predict(z)
print(test[['ID','General_Health']])
test[['ID','General_Health']].to_csv('sixth_exam.csv',index=False)

0.5617916891527253
            ID General_Health
0     ID_14547      Excellent
1        ID_26      Excellent
2     ID_14103           Good
3     ID_10946      Excellent
4      ID_1572      Excellent
...        ...            ...
4808  ID_12991      Excellent
4809  ID_13295           Good
4810   ID_2428           Fair
4811  ID_12719           Good
4812  ID_12691      Excellent

[4813 rows x 2 columns]


# 작업 3유형

## A 도시의 남성 600명과 여성 550명이 있다. 남성들 중 흡연자 비율은 0.2이며 여성들 중 흡연자 비율은 0.26이다. 남성과 여성 간에 흡연 여부에 따른 인구 비율이 다른지 확인하고 싶다. 유의 수준 0.05하 귀무가설에 대해 기각 / 채택 여부와 p-value값을 각각  출력하라 ***

In [137]:
from scipy import stats 
data = np.array([[int(600*0.2),int(600*0.8)],[int(550*0.26),int(550*0.74)]])
statistics, pvalue,dof,expect = stats.chi2_contingency(data)
print(pvalue)
if pvalue >= 0.05:
    print('귀무가설, 독립적임')
else:
    print('대립가설, 상관관계가 있음')

0.018786854975740768
대립가설, 상관관계가 있음


## 데이터 로드

In [138]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p7.csv')
df.head()

,age,Cholesterol,weight
0,65,69.0,111.0
1,54,117.0,81.0
2,61,86.2,72.0
3,57,76.0,78.0
4,62,160.0,61.0


## age와 Cholesterol을 가지고 weight를 예측하는 선형 회귀 모델을 만들려고한다. age의 회귀 계수를 구하여라

In [140]:
pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 3.9 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [141]:

import statsmodels.api as sm
X = sm.add_constant(df[['age', 'Cholesterol']]) 
model = sm.OLS(df['weight'], X)
results = model.fit()

# 전체 회귀 결과 출력
# print(results.summary())

print(results.params['age'])

-0.03610166914386492


## age가 고정일 때 Cholesterol와 weight가 선형관계에 있다는 가설을 유의수준 0.05하에 검정하라

In [145]:
H = results.pvalues['Cholesterol']

if H <0.05 : 
    print('선형 관계에 있다.')
    
else:
    print('선형 관계에 없다.')

선형 관계에 있다.


## age가 55, Cholesterol가 72.6일때 위 모델을 기반으로 weight값을 예측하라.

In [149]:
print(results.predict([1,55,72.6])[0])

78.85771011344593
